In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''

WITH
  ADC_data AS(
  SELECT
    customer_id,
    dealer_customer_id,
    BAN,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
   CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment 
  FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
    AND dealer_name='TELUS Communications Inc.' ),


Telus_customers as(

select cust_bus_cust_id,pi_cntrct_end_ts as contract_end_date,pi_cntrct_start_ts as contract_start_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-10-31' #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

),



Telus_SMHM_Deact_Oct2022 AS (

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag  
from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`
--SELECT DISTINCT BAN,1 AS Telus_Churn_Flag
--FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.ad-hoc-2023-01-25` 

),


login_data AS (
  SELECT
    customer_id,
    (COUNT(DISTINCT DATE(login_dt_utc))*100/30) AS Login_consistency
  FROM
    `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
  WHERE
    DATE(login_dt_utc)< '2022-11-01'
    AND DATE(login_dt_utc)>= '2022-10-01'
  GROUP BY
    customer_id 
),


Telus_Merge AS (


select * from ADC_data as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id 

)

, Merge_data AS (
  SELECT
    ADC.*,
    CASE
      WHEN c.Telus_Churn_Flag IS NULL THEN 0
    ELSE
    c.Telus_Churn_Flag
  END
    AS Telus_Churn,
    CASE
      WHEN lg.Login_consistency IS NULL THEN "NO_User"
      WHEN lg.Login_consistency>=30 THEN 'Heavy_User'
      WHEN lg.Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Segment
  FROM
    Telus_Merge AS ADC
  LEFT JOIN
    Telus_SMHM_Deact_Oct2022 c
  ON
    --ADC.dealer_customer_id=c.CUST_ID
    ADC.BAN=c.BAN
  LEFT JOIN
    login_data lg
  ON
    lg.customer_id=ADC.customer_id )

-- SELECT
--   Package,
--   Segment,
--   COUNT(DISTINCT customer_id) AS volume,
--   SUM(Telus_Churn) AS Total_Churn,
--   ROUND(AVG(Telus_Churn)*100,2) AS Churn_rate
-- FROM
--   Merge_data
-- GROUP BY
--   Package,
--   Segment
-- ORDER BY
--   Package,
--   Segment

select * from Merge_data



'''

In [ ]:
Query_updated='''



WITH
  ADC_data AS(
  SELECT
    customer_id,
    dealer_customer_id,
    BAN,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
   CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment 
  FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
    AND dealer_name='TELUS Communications Inc.' )


,Telus_customers as(

select cust_bus_cust_id,pi_cntrct_end_ts as contract_end_date,pi_cntrct_start_ts as contract_start_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-10-31' #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

)



,Telus_SMHM_Deact_Oct2022 AS (

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn  
from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`

)


,SHS_features as(

SELECT 
distinct ban,
AGE_BAND,
PROD_MIX_AL

 FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.base_features_scores_segments_20220731`
  
)


,Telus_Merge AS (


select ADC.*,Telus.contract_start_date,Telus.contract_end_date,Deacts.Telus_Churn,SHS.AGE_BAND,SHS.PROD_MIX_AL
from ADC_data as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join  Telus_SMHM_Deact_Oct2022 as Deacts
on ADC.BAN=Deacts.BAN
left join SHS_features as SHS
on ADC.BAN=SHS.ban

)


select * from Telus_Merge


'''

In [ ]:
Camera_count_SQL='''

SELECT customer_id,combined_device_type_desc, count(distinct device_id) as count_of_camera FROM `divgpras-pr-579355.ADC_updated.ADC_Inventory_device_data` 
where date(device_install_date)<'2022-10-01'
and date(device_removal_date) is NULL
and  combined_device_type_desc='Camera'
-- or date(device_removal_date)>='2022-11-01' 
group by customer_id,combined_device_type_desc


'''



In [ ]:
Camera_Count_DF=extract_bq_data(bq_client, sql=Camera_count_SQL)

In [ ]:
Camera_Count_DF.info()

In [ ]:
Camera_Count_DF.head()

In [ ]:
Type_of_cameras_Query= '''


SELECT distinct Customer_ID_ID as customer_id,Updated_Combined_Device_Type_DESC, count(distinct device_id) as count FROM `divgpras-pr-579355.ADC_updated.ADC_Camera_details` 
where date(Device_Install_Date_ID)<'2022-10-01'
and date(Device_Removal_Date_ID) is NULL
group by Customer_ID_ID,Updated_Combined_Device_Type_DESC



'''

In [ ]:
Telus_Smart_camera_Oct='''



with data_1 as (select * from `divgpras-pr-579355.ADC_updated.Smart_Camera_Amalysis_Oct2022`)

-- , data_2 as ( SELECT ban,AGE_BAND,PROD_MIX_AL FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.base_features_scores_segments_20220731`)


, data_2 as ( select distinct ban,prod_mix
from `bi-srv-features-pr-ef5a93.ban_product.bq_ban_product_mix`
QUALIFY ROW_NUMBER() OVER (PARTITION BY ban ORDER BY part_dt DESC) = 1)



select data_1.*,data_2.prod_mix
from data_1
left join data_2 on data_1.BAN=data_2.ban


'''

In [ ]:
Type_of_cameras=extract_bq_data(bq_client, sql=Type_of_cameras_Query)

In [ ]:
Type_of_cameras.info()

In [ ]:
Type_of_cameras['Updated_Combined_Device_Type_DESC'].value_counts()

In [ ]:
# 'Outdoor Camera'.split(" ")[0]

In [ ]:
Type_of_cameras["Device_detail_type"] = Type_of_cameras["Updated_Combined_Device_Type_DESC"].apply(lambda x: x.split(" ")[0].strip())

In [ ]:
Type_of_cameras["Device_detail_type"].value_counts()

In [ ]:
# pd.DataFrame(DF_monitored.groupby(['Arming_category','Login_category']).agg(
    
#     Customer_count= ('customer_id','count'),
#     Customer_count_1= ('customer_id','nunique'),
#     # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
#     Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
#     Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
# ).reset_index())

In [ ]:
Type_of_cameras_wide=Type_of_cameras.pivot_table(index='customer_id', columns='Device_detail_type', values='count',aggfunc = "sum").reset_index()

In [ ]:
Type_of_cameras_wide.head()

In [ ]:
Type_of_cameras_wide.columns=['customer_id','Doorbell','indoor','Outdoor']

In [ ]:
Type_of_cameras_wide.fillna(0,inplace=True)

In [ ]:
Telus_Smart_camera_Oct_DF=extract_bq_data(bq_client, sql=Telus_Smart_camera_Oct)

In [ ]:
Telus_Smart_camera_Oct_DF.info()

In [ ]:
Telus_Smart_camera_Oct_DF.isna().sum()*100/Telus_Smart_camera_Oct_DF.shape[0]

In [ ]:
Telus_Smart_camera_Oct_DF[Telus_Smart_camera_Oct_DF['BAN']==603565171]

In [ ]:
Telus_Smart_camera_Oct_DF_1=Telus_Smart_camera_Oct_DF.merge(Type_of_cameras_wide,on='customer_id',how='left')

In [ ]:
Telus_Smart_camera_Oct_DF_1.head()

In [ ]:
Telus_Smart_camera_Oct_DF_1.info()

In [ ]:
# Telus_Smart_camera_Oct_DF_1.fillna(0,inplace=True)

In [ ]:
# Telus_Smart_camera_Oct_DF_1=Telus_Smart_camera_Oct_DF_1.reset_index()

In [ ]:
Telus_Smart_camera_Oct_DF_1.head()

In [ ]:
Telus_Smart_camera_Oct_DF_1.to_csv('Telus_SmartCamera_with_camera_count_Oct2022_20230119.csv',index=False)

In [ ]:
DF=extract_bq_data(bq_client, sql=Query_updated)

In [ ]:
DF.info()

In [ ]:
DF.head()

In [ ]:
Merge_DF=DF.merge(Camera_Count_DF,how='left',on='customer_id')

In [ ]:
Merge_DF.head()

In [ ]:
Merge_DF.isna().sum()*100/Merge_DF.shape[0]

In [ ]:
Merge_DF.fillna(0,inplace=True)

In [ ]:
Merge_DF['count_of_camera'].value_counts()

In [ ]:
Merge_DF['Camera_exact_1']=Merge_DF['count_of_camera'].apply(lambda x: 1 if x==1 else 0)
Merge_DF['Camera_exact_2']=Merge_DF['count_of_camera'].apply(lambda x: 1 if x==2 else 0)
Merge_DF['Camera_grt_eq_3']=Merge_DF['count_of_camera'].apply(lambda x: 1 if x>=3 else 0)

In [ ]:
pd.DataFrame(Merge_DF.groupby(['Smart_camera_Segment','Package']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF.shape[0]),
    Churn_total=('Telus_Churn',lambda x: x.sum()),
    Churn_rate=('Telus_Churn',lambda x: x.mean()*100)).reset_index())

In [ ]:
Merge_DF.tail()

In [ ]:
Merge_DF['Package'].value_counts()

In [ ]:
pd.DataFrame(Merge_DF.groupby(['Smart_camera_Segment','Package']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF.shape[0]),
    Churn_total=('Telus_Churn',lambda x: x.sum()),
    Churn_rate=('Telus_Churn',lambda x: x.mean()*100),
    Camera_exact_1=('Camera_exact_1',lambda x: x.sum()),
    Camera_exact_2=('Camera_exact_2',lambda x: x.sum()),
    Camera_grt_eq_3=('Camera_grt_eq_3',lambda x: x.sum()),
).reset_index())

In [ ]:
Old_file_Oct=pd.read_csv('Telus_SmartCamera_with_camera_count_Oct2022_old_file.csv')

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.Smart_Camera_Amalysis_Oct2022'

bq_table_instance= bq_client.load_table_from_dataframe(Old_file_Oct, Table_BQ,job_config=config)

In [ ]:
Old_file_Oct.info()

In [ ]:
Old_file_Oct_1=Old_file_Oct.merge(Merge_DF[['customer_id','AGE_BAND','PROD_MIX_AL']],how='left',on='customer_id')

In [ ]:
Old_file_Oct_1.info()

In [ ]:
Old_file_Oct_1.head()

In [ ]:
Old_file_Oct_1.to_csv('Telus_SmartCamera_with_camera_count_Oct2022.csv',index=False)

In [ ]:
Sep_Data=pd.read_csv('Telus_SmartCamera_with_camera_count_Sep2022.csv')

In [ ]:
Oct_Data=pd.read_csv('Telus_SmartCamera_with_camera_count_Oct2022.csv')

In [ ]:
Sep_merge=Sep_Data.merge(Oct_Data[['customer_id','Segment','Telus_Churn']],how='left',on='customer_id')

In [ ]:
Sep_merge.info()

In [ ]:
Sep_merge.to_csv('Telus_SmartCamera_with_camera_count_Sep_Oct2022.csv',index=False)

In [ ]:
Customer_info=extract_bq_data(bq_client, sql=Telus_Customer_details_Sep2022)

In [ ]:
Customer_info.info()

In [ ]:
Customer_info['account_type_name'].value_counts()

In [ ]:
Customer_info['account_type_name'].value_counts(normalize=True)*100

In [ ]:
Customer_info['Telus_Churn_Flag'].fillna(0,inplace=True)

In [ ]:
pd.DataFrame(Customer_info.groupby(['account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Customer_info.shape[0]),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

In [ ]:
Login_SQL='''

with date_sql as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-10-01' and date(login_dt_utc)>= '2022-09-01' 

)



select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days, (select count_of_dates from date_sql ) as count_of_dates_logins
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-10-01' and date(login_dt_utc)>= '2022-09-01'
group by customer_id
order by number_of_login_days desc


'''

In [ ]:
Login_DF=extract_bq_data(bq_client, sql=Login_SQL)

In [ ]:
max_number_login_dates=Login_DF['count_of_dates_logins'].max()
Login_DF['Login_Consistency']=Login_DF['number_of_login_days'].apply(lambda x: x*100/max_number_login_dates)

In [ ]:
Login_DF.head()

In [ ]:
Login_DF.info()

In [ ]:
pd.DataFrame(Login_DF['Login_Consistency'].value_counts(normalize=True)*100).reset_index().sort_values(by='index',ascending=False)

In [ ]:
Merge_DF=Customer_info.merge(Login_DF,on='customer_id',how='left')

In [ ]:
Merge_DF.isna().sum()*100/Merge_DF.shape[0]

In [ ]:
Merge_DF.fillna(0,inplace=True)

In [ ]:
Merge_DF.head()

In [ ]:
def Segment_making(row,login_consistency_threshold=50):
    
    if row['Login_Consistency']>=login_consistency_threshold :
        return 'Heavy_Usage'
    elif row['Login_Consistency']==0 :
        return 'No_Usage'
    elif row['Login_Consistency']< login_consistency_threshold :
        return 'Low_Usage'
    else:
        return "Unassigned"

In [ ]:
Merge_DF['Segment']=Merge_DF.apply(Segment_making,axis=1)

In [ ]:
Merge_DF['Segment'].value_counts(normalize=True)*100

In [ ]:
def Telus_plans(row):
    if row['account_type_name'] == 'Standalone':
        return 'Smart_Camera'
    elif row['account_type_name'] =='Awareness and Automation':
        return 'Smart_Automation_Plus' 
    else:
        return 'Monitored'

Merge_DF['Telus_plans'] = Merge_DF.apply(Telus_plans, axis=1)

In [ ]:
Merge_DF['Telus_plans'].value_counts()

In [ ]:
Merge_DF['Telus_plans'].value_counts(normalize=True)*100

In [ ]:
pd.DataFrame(Merge_DF.groupby(['Telus_plans','Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF.shape[0]),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

In [ ]:
Merge_DF.to_csv('Telus_SmartCamera_Sep2022.csv',index=False)